In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import pandas as pd
import numpy as np
import gzip
import json
import math
from datetime import datetime
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

In [2]:
import sys
sys.path.append('..')
from helper.manage_orderbooks import extract_orderbooks_for_one_currencypair, load_orderbook_snapshot, plot_episode, OrderbookEpisodesGenerator
from helper.orderbook_container import OrderbookContainer
from helper.orderbook_trader import OrderbookTradingSimulator

In [3]:
filename = '/home/axel/data/small/obs_2016-11_USDT_BTC_maxVol100.dict'

# data = load_orderbook_snapshot(filename, last_line=120)

T = 4
period_length = 15
episode_windows = OrderbookEpisodesGenerator(filename=filename, episode_length=T*period_length)
print("Length of episode_windows: {}".format(len(episode_windows)))
print(len(episode_windows[0]))

Length of episode_windows: 541
60


In [4]:
data = list(episode_windows[:2])
data_cresc = list(episode_windows[17:18])

In [5]:
data[0][0]

OrderbookContainer from 2016-11-08T10:00 (factor: 1.0)
  107 bids (best: 705.0)
  110 asks (best: 705.450997)
price: 705.2254744655137
  kind: 'orderbook'
  -spread: 0.4509970000000294

In [ ]:
episode_windows[1][0]

In [ ]:
plot_episode(episode_windows[0], volume=100, intervals=4)
plot_episode(episode_windows[1], volume=100, intervals=4)

#### Test Orderbook plot, print and compare_with

In [ ]:
orderbook = data[0][0]
orderbook2 = data[0][1]

display(orderbook.head(2))
orderbook.plot(range_factor=1.02)  # outfile='sample.pdf')
print(orderbook)
print(orderbook2)

In [ ]:
orderbook = data[0][0]
orderbook2 = data[0][1]

display(orderbook.head())
display(orderbook2.head())

print("\nCompare two orderbooks:")
diff = orderbook2.compare_with(other=orderbook)

display(diff.head(10))
print(diff)

#### Test get_current_price()

In [ ]:
display(orderbook.head(3))
vol = 10

for factor in [0.25, 0.5, 0.75, 1.]:
    res = orderbook.get_current_price(vol*factor)
    print("MarketOrderPrice for {} shares: {:1.4f} (worst price: {})".format(vol*factor, res[0], res[1]))

#### Test orderbook.head and tail

In [ ]:
display(orderbook.head(2))
display(orderbook.tail(2))

#### Test parameter limit

In [ ]:
len(episode_windows[0])
from time import sleep, perf_counter as pc

In [ ]:
import time
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print(f.__name__, 'took', end - start, 'time')
        return result
    return f_timer

In [ ]:
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r args:[%r, %r] took: %2.4f sec' % \
          (f.__name__, args, kw, te-ts))
        return result
    return wrap

@timing
def f(a):
    for _ in range(a):
        i = 0
    return -1

f(100)

In [ ]:
plot_episode(data[1], limits=[710, 710], volume=100, intervals=4)

In [ ]:
plot_episode(data_cresc[0], limits=[712], volume=69.820083702999995)
plot_episode(data[0], limits=[707], volume=70.875702212199997)

In [21]:
def test_trade(data, vol, consume, limit):
    P = 15
    T = 4 
    
    ots = OrderbookTradingSimulator(orderbooks=data, volume=vol,
                                    tradingperiods=T, consume=consume, period_length=P)
    print("units:", ots.get_units_left())
    #init_shares = datasub[0].get_current_sharecount(cash=cash)
    for i in range(4):
        summary = ots.trade(limit=limit)
        print("units:", ots.get_units_left())
    
    display(ots.history)
    display(summary)
    #print(init_shares, init_shares-summary['traded'])

    return ots.history

# test = test_trade(data=data_cresc[0], vol=100, consume='volume', limit=712)
test = test_trade(data=data_cresc[0], vol=70000., consume='cash', limit=713.5)
#test_trade(data=data_cresc[0], vol=100, cash=0, consume='volume')

units: 70000.0
units: 27139.54
units: 18560.41
units: 18560.41
units: 0.0


,ASK,BID,CASH,CENTER,LIMIT,SPREAD,T,VOLUME,avg,cash_traded,cost,extra_shares,forced,high,initial_marketShares,low,slippage,volume_traded
2016-11-09T03:01,711.419498,709.740242,70000.00,710.579539,713.5,1.679257,15,0.000000,712.08015,-42860.46,-365.931420,0.514976,False,713.476643,97.462479,711.419497,90.322511,60.190500
2016-11-09T03:16,712.519499,711.855642,27139.54,712.187519,713.5,0.663858,15,-60.190500,712.90490,-8579.13,-63.342213,0.089142,False,713.415999,97.462479,712.519499,27.983497,12.034046
2016-11-09T03:31,713.800000,712.978158,18560.41,713.389000,713.5,0.821842,15,-72.224545,0.00000,0.00,0.000000,0.000000,False,0.000000,97.462479,inf,NaN,0.000000
2016-11-09T03:46,713.857091,717.769723,18560.41,715.811625,713.5,-3.912632,15,-72.224545,721.53289,-18560.41,84.181928,-0.118469,True,724.384442,97.462479,713.857091,281.759411,25.723581


{'cash_traded': -70000.0,
 'cost': -345.0917053857649,
 'done': True,
 'extra_shares': 0.48564824389999994,
 'slippage': 400.06541909173262,
 'volume_traded': 97.948126779600017}

In [ ]:
test = test_trade(data=data_cresc[0], vol=71904.78, consume='cash', limit=712)

In [ ]:
data[1][0].get_current_sharecount(cash=50000)

In [ ]:
print(hist[['ASK', 'BID', 'CENTER', 'LIMIT', 'VOLUME', 'avg', 'cost', 'forced', 'high', 'volume_traded']].to_latex(float_format="%.3f"))

In [ ]:
P = 2
vol=100
T = 4
ots = OrderbookTradingSimulator(orderbooks=episode_windows[0][:T*P], volume=vol, tradingperiods=T,
                                period_length=P)

for e in tqdm(range(T)):
    summary = ots.trade(limit=706, verbose=True)

display(ots.history)
display(summary)


#### Test parameter aggression_factor

In [ ]:
P = 2
vol=100
T = 4
ots = OrderbookTradingSimulator(orderbooks=episode_windows[0][:T*P], volume=vol, tradingperiods=T,
                                period_length=P)

for e in tqdm(range(T)):
    action=0.9
    summary = ots.trade(agression_factor=action, verbose=True, extrainfo={'ACTION':action})
display(ots.history)